Урок 8. Python применение статистических критериев на практике - пайплайн оценки A/B
На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [1]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
import pandas as pd
import scipy.stats as st 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

In [19]:
df = pd.read_csv('ab_statistic.csv')

In [20]:
df.describe(include='all')

,USER_ID,VARIANT_NAME,REVENUE
count,10000.000000,10000,10000.000000
unique,NaN,2,NaN
top,NaN,variant,NaN
freq,NaN,5016,NaN
mean,4981.080200,NaN,0.099447
std,2890.590115,NaN,2.318529
min,2.000000,NaN,0.000000
25%,2468.750000,NaN,0.000000
50%,4962.000000,NaN,0.000000
75%,7511.500000,NaN,0.000000


In [26]:
# Очистка данных
# Заменить 'variant' на 'A', 'control' на 'B', удалить все остальные значения
df['VARIANT_NAME'] = np.where(df['VARIANT_NAME'] == 'variant', 'A', df['VARIANT_NAME'])
df['VARIANT_NAME'] = np.where(df['VARIANT_NAME'] == 'control', 'B', df['VARIANT_NAME'])

# Удалить строки, не содержащие 'A' или 'B' в 'VARIANT_NAME'
df = df[df['VARIANT_NAME'].isin(['A', 'B'])]

In [28]:
# Удалить дубликаты по 'USER_ID'
df.drop_duplicates(subset='USER_ID', inplace=True)

In [42]:
df.head()

USER_ID         90
VARIANT_NAME    90
 REVENUE        90
dtype: int64